# Model estimation

In [ ]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.utils.np_utils import to_categorical

import sys,os

def get_parent_dir(directory):
    import os
    return os.path.dirname(directory)

def make_digit(word):
    if word == 'positive':
        return 0
    elif word == 'neutral':
        return 1
    elif word == 'negative'
        return 2
    else:
        '{} is neither positive, neutral or negative'.format()

current_dirs_parent = get_parent_dir(os.getcwd())

### NOTE: IMPROVE!!

here I just turn the vocabulary to 10,000 values, but we should rather keep the 10,000 top words (should count how many are distinct) and let the rest take value unknown, maybe?

or just increase V?

In [230]:
sent_num_label = {}
sent_hashed = list()
sent_label = list()
max_sentence_length = 0
with open(current_dirs_parent + '/data/OLDFilteredSentences_labels.csv', 'rb') as labels: 
    for line in labels:
        num, label = line.strip().split(',')
        sent_num_label[num] = label.lower()
with open(current_dirs_parent + '/data/OLDFilteredSentences.txt','rb') as sentences:
    for i, line in enumerate(sentences):
            (key, val) = line.split("']")
            # maybe Hyera updated her sentence labels and removed some of them?
            try:
                sent_label.append(make_digit(sent_num_label[str(i+1)]))
                max_sentence_length = np.max([max_sentence_length, len(val.split(' '))])
                sent_hashed.append(text.one_hot(val, V))
            except KeyError:
                pass

### Train-test split e.g., on 75%-25%, and we should cross validate but we have so few sentences, that i just wanted to check whether it works

In [231]:
from random import shuffle
ids = list(range(701))
shuffle(ids)
P_TRAIN = .75
N_TRAIN = int(round(P_TRAIN * len(sent_label),0))
train_ids = ids[:N_TRAIN]
test_ids = ids[(N_TRAIN + 1):]
train_sents, train_labels = zip(*[(sent_hashed[train_id], sent_label[train_id]) for train_id in train_ids])
test_sents, test_labels = zip(*[(sent_hashed[test_id], sent_label[test_id]) for test_id in test_ids])

# add padding so that all of the sentences have the same length
train_sents = sequence.pad_sequences(train_sents, maxlen=max_sentence_length)
test_sents = sequence.pad_sequences(test_sents, maxlen=max_sentence_length)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
# create the model
embedding_vecor_length = 300
model = Sequential()
model.add(Embedding(V, embedding_vecor_length, input_length=max_sentence_length))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(train_sents, train_labels, nb_epoch=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_sents, test_labels, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 141, 300)          3000000   
_________________________________________________________________
lstm_21 (LSTM)               (None, 100)               160400    
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 303       
Total params: 3,160,703
Trainable params: 3,160,703
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
526/526 [==============================] - 4s - loss: 1.0315 - acc: 0.6540     
Epoch 2/3
526/526 [==============================] - 3s - loss: 0.7241 - acc: 0.7129     
Epoch 3/3
526/526 [==============================] - 3s - loss: 0.6166 - acc: 0.7129     
Accuracy: 67.24%


it seems that it predicts only neutral (second column has the max. value) below

In [232]:
model.predict(test_sents)

array([[ 0.17751227,  0.77461505,  0.04787262],
       [ 0.23319693,  0.66797918,  0.09882392],
       [ 0.24523947,  0.67335522,  0.08140534],
       [ 0.18965013,  0.75048065,  0.05986925],
       [ 0.12022069,  0.85089761,  0.02888172],
       [ 0.19153422,  0.75297087,  0.05549492],
       [ 0.09722652,  0.88207144,  0.02070205],
       [ 0.09107928,  0.89431864,  0.01460207],
       [ 0.0840079 ,  0.89915258,  0.01683953],
       [ 0.22692826,  0.68821895,  0.08485274],
       [ 0.15048903,  0.80914021,  0.04037081],
       [ 0.0896502 ,  0.89265764,  0.01769217],
       [ 0.1078333 ,  0.86824173,  0.02392489],
       [ 0.13480806,  0.8295399 ,  0.03565199],
       [ 0.24974583,  0.62930715,  0.12094695],
       [ 0.25039676,  0.66958117,  0.08002204],
       [ 0.13957691,  0.82589972,  0.03452346],
       [ 0.22364967,  0.67058307,  0.1057673 ],
       [ 0.21047632,  0.70749813,  0.08202552],
       [ 0.21663295,  0.72615677,  0.05721035],
       [ 0.2010127 ,  0.74366742,  0.055